# IMPORTING REQUIRED LIBRARIES

In [2]:
from selenium import webdriver                          #For opening browser       
import pandas as pd                                     #For creating dataframe
from bs4 import BeautifulSoup                           #For scraping data
import time                                             #For waiting
import numpy as np                                      #For array
from tqdm import tqdm                                   #For progress bar

# OPENING REQUIRED SITE IN BROWSER

In [3]:
driver = webdriver.Firefox()

In [4]:
driver.get('https://m.imdb.com/chart/starmeter/')

# GETTING HTML CODE OF OPENED WEBPAGE

In [5]:
html = driver.page_source

# CONVERTING EXTRACTED HTML CODE INTO SOUP TO SCRAP DATA

In [6]:
soup = BeautifulSoup(html,"html.parser")

# EXTRACTING NAMES OF CELEBRITIES AND LINKS TO THEIR INDIVIDUAL WEBPAGES

In [7]:
a_list = soup.find_all("a")
name_list = []
link_list = []
for i in range(43,343,3):
    name_list.append(a_list[i].get('aria-label'))
    link_list.append(a_list[i].get('href'))

# EXTRACTING THE FAMOUS WORK OF THESE CELEBRITIES

In [8]:
famous_list = []
a_list1 = soup.findAll('a',{'data-testid':'nlib-known-for-title'})
for i in a_list1:
    famous_list.append(i.text)

# FUNCTION TO EXTRACT DATE OF BIRTH OF THE CELEBRITIES

In [9]:
def get_dob(soup):
    try:
        sec = soup.findAll('section', {'data-testid':'PersonalDetails'})[0]
        dob = sec.findAll('li',{'data-testid':'nm_pd_bl'})[0].findAll('li')[0].text
    except Exception as e:
        dob = np.nan
    return dob

# FUNCTION TO EXTRACT HEIGHT OF THE CELEBRITIES

In [10]:
def get_height(soup):
    try:
        sec = soup.findAll('section', {'data-testid':'PersonalDetails'})[0]
        height = sec.findAll('li',{'data-testid':'nm_pd_he'})[0].findAll('li')[0].text
    except Exception as e:
        height = np.nan
    return height

# FUNCTION TO EXTRACT BIRTHPLACE OF THE CELEBRITIES

In [11]:
def get_birthplace(soup):
    try:
        sec = soup.findAll('section', {'data-testid':'PersonalDetails'})[0]
        bp = sec.findAll('li',{'data-testid':'nm_pd_bl'})[0].findAll('li')[1].text
    except Exception as e:
        bp = np.nan
    return bp

# FUNCTION TO EXTRACT AWARD DATA OF THE CELEBRITIES

In [12]:
def get_awards(soup):
    try:
        div_list = soup.findAll('div', {'data-testid':'awards'})
        awards = div_list[0].findAll('li',{'class':'ipc-inline-list__item'})[0].text
    except Exception as e:
        awards = np.nan
    return awards

# FUNCTION TO EXTRACT ROLES OF THE CELEBRITIES

In [13]:
def get_role(soup):
    role_list = []
    try:
        li_list = soup.findAll('ul', {'class':'ipc-inline-list ipc-inline-list--show-dividers sc-d8941411-2 cdJsTz baseAlt'})[0].findAll('li')
        for i in li_list:
            role_list.append(i.text)
    except Exception as e:
        role_list.append(np.nan)
    return role_list

In [14]:
get_role(soup)

[nan]

# MAIN LOOP TO ITERATE THROUGH EACH CELEBRITY'S WEBPAGE AND EXTRACT DATA

In [15]:
dob_list = []
height_list = []
role_list = []
award_list = []
birth_list = []
for i in tqdm(link_list, desc="Processing Links"):
    driver.get(f'https://m.imdb.com/{i}')
    #time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html,"html.parser")
    dob_list.append(get_dob(soup))
    height_list.append(get_height(soup))
    birth_list.append(get_birthplace(soup))
    role_list.append(get_role(soup))
    award_list.append(get_awards(soup))

Processing Links:   0%|          | 0/100 [00:00<?, ?it/s]

Processing Links: 100%|██████████| 100/100 [14:57<00:00,  8.97s/it]


# CREATING DATAFRAME FOR COLLECTED DATASET 

In [16]:
df = pd.DataFrame({
    'Rank':np.arange(1,101),
    'Name': name_list,
    'Date_of_birth': dob_list,
    'height': height_list,
    'Role': role_list,
    'Awards': award_list,
    'Famous_for': famous_list,
    'Birth_place': birth_list
})

# CHECKING THE DATASET

In [17]:
df.head()

,Rank,Name,Date_of_birth,height,Role,Awards,Famous_for,Birth_place
0,1,Andre Braugher,"July 1, 1962",6′ (1.83 m),"[Actor, Producer, Director]",15 wins & 70 nominations total,Homicide: Life on the Street (1993–1998),"Chicago, Illinois, USA"
1,2,Julia Roberts,"October 28, 1967",5′ 8″ (1.73 m),"[Actress, Producer, Soundtrack]",61 wins & 115 nominations total,Pretty Woman (1990),"Smyrna, Georgia, USA"
2,3,Myha'la,"April 6, 1996",5′ 1″ (1.55 m),[Actress],NaN,Industry (2020–2022),"California, USA"
3,4,Serinda Swan,"July 11, 1984",5′ 7″ (1.70 m),"[Actress, Producer, Director]",2 nominations,Coroner (2019–2022),"West Vancouver, British Columbia, Canada"
4,5,Shelley Hennig,"January 2, 1987",5′ 8″ (1.73 m),[Actress],1 win & 4 nominations,Unfriended (2014),"Destrehan, Louisiana, USA"


In [18]:
df.tail()

,Rank,Name,Date_of_birth,height,Role,Awards,Famous_for,Birth_place
95,96,Joel Kinnaman,"November 25, 1979",6′ 2½″ (1.89 m),"[Actor, Producer]",1 win & 5 nominations,The Killing (2011–2014),"Stockholm, Sweden"
96,97,Tatum O'Neal,"November 5, 1963",5′ 7″ (1.70 m),"[Actress, Producer, Soundtrack]",7 wins & 6 nominations total,Paper Moon (1973),"Los Angeles, California, USA"
97,98,Kevin Bacon,"July 8, 1958",5′ 10″ (1.78 m),"[Actor, Producer, Director]",16 wins & 28 nominations total,Footloose (1984),"Philadelphia, Pennsylvania, USA"
98,99,Nicolette Scorsese,"January 6, 1954",NaN,[Actress],NaN,National Lampoon's Christmas Vacation (1989),NaN
99,100,Zac Efron,"October 18, 1987",5′ 8″ (1.73 m),"[Actor, Producer, Soundtrack]",35 wins & 42 nominations,17 Again (2009),"San Luis Obispo, California, USA"


In [19]:
df.sample(20)

,Rank,Name,Date_of_birth,height,Role,Awards,Famous_for,Birth_place
55,56,Vanessa Hudgens,"December 14, 1988",5′ 1¼″ (1.55 m),"[Actress, Producer, Soundtrack]",12 wins & 23 nominations,High School Musical 3: Senior Year (2008),"Salinas, California, USA"
15,16,Noah LaLonde,"March 22, 1998",NaN,"[Actor, Producer]",NaN,My Life with the Walter Boys (2023– ),NaN
22,23,Juno Temple,"July 21, 1989",5′ 2″ (1.57 m),"[Actress, Soundtrack]",6 wins & 22 nominations total,Atonement (2007),"London, England, UK"
67,68,Alex Garland,"May 26, 1970",6′ 1″ (1.85 m),"[Writer, Producer, Director]",22 wins & 69 nominations total,Ex Machina (2014),"London, England, UK"
18,19,Willa Fitzgerald,"January 17, 1991",5′ 5″ (1.65 m),"[Actress, Soundtrack]",4 nominations,Scream: The TV Series (2015–2016),"Nashville, Tennessee, USA"
32,33,Lily Gladstone,"August 2, 1986",5′ 7″ (1.70 m),"[Actress, Casting Department, Writer]",27 wins & 37 nominations,Certain Women (2016),"Browning, Montana"
94,95,Judge Reinhold,May 21,6′ 2″ (1.88 m),"[Actor, Producer, Director]",1 nomination total,Beverly Hills Cop (1984),"Wilmington, Delaware, USA"
51,52,Thomas Brodie-Sangster,"May 16, 1990",5′ 10½″ (1.79 m),"[Actor, Producer, Soundtrack]",4 wins & 12 nominations total,Maze Runner: The Scorch Trials (2015),"London, England, UK"
93,94,Anna Sawai,June 11,5′ 1″ (1.55 m),[Actress],1 win & 1 nomination,Ninja Assassin (2009),New Zealand
41,42,Nichole Sakura,"December 15, 1989",5′ 7″ (1.70 m),"[Actress, Writer, Director]",2 wins,Until Dawn (2015),"Santa Clara County, California, USA"


# SAVING THE DATASET

In [20]:
df.to_csv('celebrity_data.csv',index= False)